# Langchain Integration into Azure Machine Learning (AzureML)

**Requirements** - In order to benefit from this tutorial, you will need:
* A basic understanding of Machine Learning and Large Language Models
* An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
* An Azure Machine Learning Workspace, Azure Key Vault, and Azure Container Registry
* An OpenAI API Key which can be found in User Settings in OpenAI

**Motivations** - The Langchain framework allows for rapid development of applications powered by large language models. This sample creates a chat bot application backed by a large language model and deploys the application to AzureML.

**Outline** - 
1. Test langchain app locally using [**azureml-inference-server-http**](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-inference-server-http?view=azureml-api-2) pacakge.
2. Deploy the app to an **AzureML Managed Online Endpoint**


# 1. Setup Variables
Before deploy, you could test the langchain app locally. We are using [Langchain ChatGPT plugin](https://python.langchain.com/en/latest/modules/agents/tools/examples/chatgpt_plugins.html) as an example app here. Execute the code below to try out. You can inspect the [simple_agent_app_test.py](../src/langchain/simple_agent_app_test.py) to see the implementation itself. It's a langchain ZERO_SHOT_REACT_DESCRIPTION agent with Klarna plugin.

In [ ]:
OPENAI_API_TYPE = "openai"  # 'azure' or 'openai'
OPENAI_API_KEY = "<OPENAI_API_KEY>"

# required for OpenAI API
OPENAI_ORG_ID = ""
OPENAI_MODEL_ID = "gpt-3.5-turbo"

# required for Azure OpenAI API
AZURE_OPENAI_API_ENDPOINT = "<AOAI endpoint>"
AZURE_OPENAI_API_DEPLOYMENT_NAME = "<deployment-name>"

# set to env var for the langchain code to consume
%env OPENAI_API_KEY=$OPENAI_API_KEY
%env OPENAI_API_TYPE=$OPENAI_API_TYPE
%env OPENAI_MODEL_ID=$OPENAI_MODEL_ID
%env OPENAI_ORG_ID=$OPENAI_ORG_ID
%env AZURE_OPENAI_API_ENDPOINT=$AZURE_OPENAI_API_ENDPOINT
%env AZURE_OPENAI_API_DEPLOYMENT_NAME=$AZURE_OPENAI_API_DEPLOYMENT_NAME

In [ ]:
%pip install -r requirements.txt

# 2. Deploy Langchain App to an Online Endpoint

## 2.1 Preparation

In [ ]:
# enter details of your AML workspace
SUBSCRIPTION_ID = "<SUBSCRIPTION_ID>"
RESOURCE_GROUP = "<RESOURCE_GROUP>"
AML_WORKSPACE_NAME = "<AML_WORKSPACE_NAME>"

Login to your account

In [ ]:
# Authenticate clients
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential(additionally_allowed_tenants=["*"])
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential(additionally_allowed_tenants=["*"])

# If login doesn't work above, uncomment the code below and login using device code
# !az login --use-device-code

# 3. Manage Online Endpoint
## 3.1 Create Endpoint

In [ ]:
# create a endpoint
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
)

from azure.ai.ml import (
    MLClient,
)

online_endpoint_name = "aml-llm-demo-langchain-endpoint"

# get a handle to the workspace
ml_client = MLClient(credential, SUBSCRIPTION_ID, RESOURCE_GROUP, AML_WORKSPACE_NAME)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="online endpoint for Langchain server",
    auth_mode="key",
)

endpoint = ml_client.begin_create_or_update(endpoint).result()

print(endpoint)

## 3.2 Deploy to Endpoint

In [ ]:
import datetime

from azure.ai.ml.entities import (
    ManagedOnlineDeployment,
    OnlineRequestSettings,
    Environment,
    CodeConfiguration,
)

env = Environment(
    conda_file="deployments/env.yml",
    image="mcr.microsoft.com/azureml/minimal-ubuntu20.04-py38-cpu-inference:latest",
)

deployment_name = f"deploy-{str(datetime.datetime.now().strftime('%m%d%H%M%f'))}"
lc_deployment = ManagedOnlineDeployment(
    name=deployment_name,
    environment=env,
    code_configuration=CodeConfiguration(
        code="../src", scoring_script="langchain/simple_agent_score.py"
    ),
    request_settings=OnlineRequestSettings(request_timeout_ms=60000),
    environment_variables={
        "OPENAI_API_KEY": OPENAI_API_KEY,
        "OPENAI_API_TYPE": OPENAI_API_TYPE,
        "OPENAI_MODEL_ID": OPENAI_MODEL_ID,
        "OPENAI_ORG_ID": OPENAI_ORG_ID,
        "AZURE_OPENAI_API_ENDPOINT": AZURE_OPENAI_API_ENDPOINT,
        "AZURE_OPENAI_API_DEPLOYMENT_NAME": AZURE_OPENAI_API_DEPLOYMENT_NAME,
    },
    endpoint_name=online_endpoint_name,
    instance_type="Standard_F2s_v2",
    instance_count=1,
)
ml_client.online_deployments.begin_create_or_update(lc_deployment).result()

endpoint.traffic = {deployment_name: 100}
ml_client.begin_create_or_update(endpoint).result()

# 4. Test
Now endpoint has been deployed, let's test it. We are going to re-use the same request when we test it locally earlier on.

In [ ]:
import requests, json
from urllib.parse import urlsplit

url_parts = urlsplit(endpoint.scoring_uri)
url = url_parts.scheme + "://" + url_parts.netloc

token = ml_client.online_endpoints.get_keys(name=online_endpoint_name).primary_key
headers = {"Authorization": "Bearer " + token, "Content-Type": "application/json"}
payload = json.dumps(
    {"question": "what are the top 5 results for womens t shirts on klarna?"}
)

response = requests.post(f"{url}/score", headers=headers, data=payload)
print(f"Response:\n", response.text)

# 5. Delete the deployment and endpoint

In [ ]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)